In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Read the vocabulary of words for names
with open('names.txt', 'r') as f:
    words = f.read().splitlines()

In [3]:
len(words)

32033

In [9]:
# build vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [11]:
# Build the dataset
block_size = 3    # context length - how many characters do we take to predict the next one
X, Y = [], []

for w in words[:5]:
    print(f"word is: {w}")
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(f"X is: {X}::Y is: {Y}")
        
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
        print(f"Updated Context is: {context}")

word is: emma
X is: [[0, 0, 0]]::Y is: [5]
... ---> e
Updated Context is: [0, 0, 5]
X is: [[0, 0, 0], [0, 0, 5]]::Y is: [5, 13]
..e ---> m
Updated Context is: [0, 5, 13]
X is: [[0, 0, 0], [0, 0, 5], [0, 5, 13]]::Y is: [5, 13, 13]
.em ---> m
Updated Context is: [5, 13, 13]
X is: [[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13]]::Y is: [5, 13, 13, 1]
emm ---> a
Updated Context is: [13, 13, 1]
X is: [[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1]]::Y is: [5, 13, 13, 1, 0]
mma ---> .
Updated Context is: [13, 1, 0]
word is: olivia
X is: [[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0]]::Y is: [5, 13, 13, 1, 0, 15]
... ---> o
Updated Context is: [0, 0, 15]
X is: [[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15]]::Y is: [5, 13, 13, 1, 0, 15, 12]
..o ---> l
Updated Context is: [0, 15, 12]
X is: [[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12]]::Y is: [5, 13, 13, 1, 0, 15, 12, 9]
.ol 

In [ ]:
X = torch.tensor(X)
Y = torch.tensor(Y)